In [1]:
# installiamo le librerie necessarie
!pip install --upgrade --quiet langchain
!pip install --upgrade --quiet langchain_openai
!pip install --upgrade --quiet langchain_chroma
!pip install --upgrade --quiet langchain_community

In [2]:
# Carichiamo l'API Key di OpenAI
import os
os.environ['OPENAI_API_KEY'] = 'xxx'

In [3]:
# Usiamo il Document Loader per caricare il video
from langchain_community.document_loaders import TextLoader
loader = TextLoader("./i_promessi_sposi.txt")
documents = loader.load()

In [4]:
# Il nostro documento è troppo grande per entrare nella context windows di GPT
# Usiamo un text spliltter per dividerlo in documenti più piccoli
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=5000,
)
documents = text_splitter.split_documents(documents)
print(f"Diviso il testo in {len(documents)} documenti")

In [6]:
# Usiamo il prompt template di langchain
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
	("user", """
Utilizza il contenuto del testo fornito per rispondere alla domanda dell'utente. Rispondi che non conosci la risposta se non è contenuta nel testo.
Domanda: {question}
	 
Testo:
{text}
""")
])

# Ora possiamo invocare GPT per generare il riassunto.
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# riscriviamo l'ultima parte usando le chain
from langchain_core.runnables import RunnablePassthrough
chain = ({"text": RunnablePassthrough(), "question": RunnablePassthrough()} | prompt_template | llm)

In [7]:
# Inserisci una domanda relativa al contenuto del testo
question = "Chi era Don Abbondio?"

# Eseguiamo la chain
answer = chain.invoke({"text": documents[0].page_content, "question": question})

# Stampiamo il risultato
print(answer.content)

Don Abbondio era il curato di una delle terre accennate nel testo. Non si conosce il nome specifico di questa terra né il casato del personaggio, poiché non sono presenti nel manoscritto.
